In [1]:
import numpy as np
import pandas as pd
import pyhere

In [2]:
dir_raw_data = pyhere.here().resolve().joinpath("data", "raw")


In [3]:
csv_power_plants = pd.read_csv(dir_raw_data/"global_power_plant_database.csv")

/tmp/ipykernel_1438/1573865454.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_power_plants = pd.read_csv(dir_raw_data/"global_power_plant_database.csv")


In [113]:
sample_lat_lon = csv_power_plants[["latitude", "longitude"]].head(2)


In [114]:
list_of_tuples = list(sample_lat_lon.to_records(index=False))
list_of_tuples


[(32.322, 65.119), (31.67, 65.795)]

str

# LOOP SINGLE POINT

In [125]:
'''
*Version: 2.0 Published: 2021/03/09* Source: [NASA POWER](https://power.larc.nasa.gov/)
POWER API Multi-Point Download
This is an overview of the process to request data from multiple data points from the POWER API.
'''
# TQV                   MERRA-2 Total Column Precipitable Water (kg m-2) 
# WS10M                 MERRA-2 Wind Speed at 10 Meters (m/s) 
# CLRSKY_SFC_SW_DNI     CERES SYN1deg Clear Sky Surface Shortwave Downward Direct Normal Irradiance (kW-hr/m^2/day) 


import os, json, requests

locations = list_of_tuples
north_hemisfere = list()
south_hemisfere = list()
north_hemisfere["autumn"] = ("OCT", "NOV", "DEC")
north_hemisfere["winter"] = ("JAN", "FEB", "MAR")
north_hemisfere["spring"] = ("APR", "MAY", "JUN")
north_hemisfere["summer"] = ("JUL", "AUG", "SEP")
south_hemisfere["spring"] = ("OCT", "NOV", "DEC")
south_hemisfere["summer"] = ("JAN", "FEB", "MAR")
south_hemisfere["autumn"] = ("APR", "MAY", "JUN")
south_hemisfere["winter"] = ("JUL", "AUG", "SEP")

output = r""
base_url = r"https://power.larc.nasa.gov/api/temporal/monthly/point?parameters=CLRSKY_SFC_SW_DNI,WS10M,TQV&community=RE&longitude={longitude}&latitude={latitude}&start=1990&end=2020&format=CSV&header=false"
df_response = pd.DataFrame()


for latitude, longitude in locations:
    api_request_url = base_url.format(longitude=longitude, latitude=latitude)

    
    response = requests.get(url=api_request_url, verify=False, timeout=30.00).content.decode('utf-8')

    # content = json.loads(response.content.decode('utf-8'))
    # filename = response.headers['content-disposition'].split('filename=')[1]
    filename_template = "File_Lat_{latitude}_Lon_{longitude}.csv"
    filename = filename_template.format(latitude=latitude, longitude=longitude)

    # print(pd.read_csv(StringIO(response)))
    df_response_aux = pd.read_csv(StringIO(response))
    df_response_aux["latitude"] = latitude
    df_response_aux["longitude"] = longitude
    # print(type(df_response_aux))
    # print(df_response.empty)
    # df_response_aux.to_csv(f"df_response_aux_{filename}")
    if(df_response.empty):
        
        # print("sasa1")
        df_response = df_response_aux.copy()
        # print(df_response)
        # df_response.to_csv(f"df_response_{filename}")
    else:
        # print("sasa2")
        df_response = pd.concat([df_response,df_response_aux], ignore_index = True)
        # print(df_response)
        # df_response.to_csv(f"df_response_sasa_{filename}")
        
        # df_response_aux.to_csv(f"df_response_aux_sasa_{filename}")
    
# print(df_response)
df_response.to_csv(filename)
# with open(filename, 'w') as file_object:
#     file_object.write(response)
#     file_object.close()